In [62]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits import mplot3d
from scipy.spatial import *
import pandas as pd

import os
os.system('ls *.pdb > pdb.list')
#generate a list with all pdbs

0

In [29]:
def readin_files(file):
    data_list = []
    with open(file) as open_file:
        for line in open_file:
            data_list.append(line[:-1])
    return data_list
#defines the reading of files

model = readin_files('pdb.list')
volume_residues = readin_files('selected_res_vol.list')
bw_numbering = readin_files('ADRB1.bw')
#read in all the necessary files and assignments

relevant_vol_res = []
for residue in bw_numbering:
    res = residue.split()
    if res[2] in volume_residues:
        relevant_vol_res.append(res[1])
print(relevant_vol_res)
#build a Rosetta number list


['63', '71', '86', '95', '176', '185', '292', '299', '308', '319']


In [47]:
for structure in model:
    xyz_name = f"{structure}.xyz"
    xyz_file = open(xyz_name, 'w')
    for lines in open(structure, 'r').readlines():
        line = lines.split()
        try:
            if line[5] in relevant_vol_res and line[4] == 'A' and line[2] == 'CA': 
                print(*line[3:9], sep='\t', file=xyz_file)
        except:
            pass
    xyz_file.close()
#selects from all pdbs the relevant residues (CA atom) and the x, y and z coordinates

In [60]:
xyz = []
for structure in model:
    xyz.append((structure + ".xyz"))

counter = 0
for ele in xyz:
    vol_file = open(f"{ele}.vol", "w")
    points = []
    file = open(ele, 'r')
    try:
        for line in file:
            line = line.split()
            if "NA" not in line:
                point = line[-3:]
            points.append(point)
        hull = ConvexHull(points)

        print(ele, hull.volume, file=vol_file)
        vol_file.close()
        counter += 1
        print(counter)
    except:
        print(ele, "NA", file=f)
#For calculating the volume of the 10 res and saving them in a new file
os.system('find . -name "*vol" -exec cat {} > all_vol.list \;')
#One final file for all volumes
os.system('find . -name "*.pdb" | xargs grep -E "pose" > score.list \;')

1
2
3
4
5
6
7
8
9
10


31488

In [86]:
df_volume = pd.read_csv('all_vol.list', names=['description', 'volume'], sep='\s+')
df_volume['description'] = df_volume['description'].map(lambda x: str(x)[:-4])
df_score = pd.read_csv('score.list', header=None, sep='\s+')
df_score['description'] = df_score[0].map(lambda x: str(x)[2:-5])
df_score_final = df_score[[23, 'description']]
merged = pd.merge(left=df_volume, right=df_score_final, on='description')
merged = merged.rename(columns={23: 'score'})
merged
#read in the volume and the score files and generates a merged dataframe

,description,volume,score
0,10_S_0001.pdb,1968.550541,-1203.25
1,1_S_0001.pdb,2338.346394,-1304.82
2,2_S_0001.pdb,2128.203834,-1283.98
3,3_S_0001.pdb,2583.798224,-1206.92
4,4_S_0001.pdb,2522.088623,-1259.55
5,5_S_0001.pdb,2504.961395,-1206.16
6,6_S_0001.pdb,2600.626158,-1208.19
7,7_S_0001.pdb,2468.116088,-1248.59
8,8_S_0001.pdb,2374.113802,-1253.85
9,9_S_0001.pdb,2331.991950,-1166.50


In [98]:
merged['volume'] = merged['volume'].astype(float)
merged['score'] = merged['score'].astype(float)
mean_vol = merged['volume'].mean()

print(mean_vol)
selected = merged.loc[(merged['volume'] > (mean_vol-200)) & (merged['volume'] < (mean_vol+200))]
selected.sort_values(by=['score'], inplace=True, ascending=True)
selected.to_csv('within_200.list', sep='\t', index=None)
os.system(f'echo "\nThe mean_volume was {round(mean_vol, 2)}" >> within_200.list')
best5 = selected.head(5)
best5.to_csv('best5.list', sep='\t', index=None)
os.system(f'echo "\nThe mean_volume was {round(mean_vol, 2)}" >> best5.list')

2382.0797008176332


0